In [1]:
import os
import sys
import time
import json
import re
import pandas as pd
import numpy as np
from lxml import etree as ET
from collections import OrderedDict

## Scraping the NSF
Use BeautifulSoup to crawl the NSF site and extract awards, award recipients (PI's), univerities and NSF internal divisions. Dump'em into JSON for analysis/warehousing...
<br>Currently only one year at a time-- how to scale up + do it all in memory?(!?!)
<br>TODO: use multiprocessing pool to speed up crawls!
<br><t> cut doops by checking if key exists

Notes from Ivan:
Use Lxml rather than b4
don't fund before use
Classes!
Declarative

In [2]:
tree = ET.parse('data/2010/1000018.xml')

In [3]:
root = tree.getroot()
for child1 in root:
    print(child1.tag)
    for child2 in child:
        print("\t",child2.tag)
        for child3 in child2:
            print("\t\t",child3.tag)

Award
Traceback (most recent call last):
  File "/Users/leonyin/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 3066, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-37fff6ff9fe7>", line 4, in <module>
    for child2 in child:
NameError: name 'child' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/leonyin/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 1877, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/leonyin/anaconda/lib/python3.5/site-packages/IPython/core/ultratb.py", line 970, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/Users/leonyin/anaconda

ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.


Unfortunately, your original traceback can not be constructed.



TypeError: 'NoneType' object is not iterable

In [4]:
tree.find('Award/Organization/Division')[0].text

'Div Of Civil, Mechanical, & Manufact Inn'

In [ ]:
def find2():
    start = time.time()
    OrderedDict({tree.find('Award/AwardID').tag:tree.find('Award/AwardID').text,
                 tree.find('Award/AwardTitle').tag:tree.find('Award/AwardTitle').text,
                 tree.find('Award/AwardEffectiveDate').tag:tree.find('Award/AwardEffectiveDate').text,
                 tree.find('Award/AwardExpirationDate').tag:tree.find('Award/AwardExpirationDate').text,
                 tree.find('Award/AwardAmount').tag:tree.find('Award/AwardAmount').text,
                 tree.find('Award/Institution/Name').tag:tree.find('Award/Institution/Name').text,
                 tree.find('Award/Organization/Division').tag:tree.find('Award/Organization/Division').text[0],
                 tree.find('Award/Investigator/EmailAddress').tag:tree.find('Award/Investigator/EmailAddress').text})
    return((time.time()-start)*1e6)

In [ ]:
tree.xpath('Award/AwardID')[0].tag

In [ ]:
def find1():
    start = time.time()
    OrderedDict({'Award/AwardID' : tree.find('Award/AwardID').text,
                 'Award/AwardTitle' : tree.find('Award/AwardTitle').text,
                 'Award/AwardEffectiveDate' : tree.find('Award/AwardEffectiveDate').text,
                 'Award/AwardExpirationDate':tree.find('Award/AwardExpirationDate').text,
                 'Award/AwardAmount':tree.find('Award/AwardAmount').text,
                 'Award/Institution/Name' :tree.find('Award/Institution/Name').text,
                 'Award/Organization/Division' : tree.find('Award/Organization/Division')[0].text,
                 'Award/Investigator/EmailAddress' : tree.find('Award/Investigator/EmailAddress').text})
    return((time.time()-start)*1e6)

In [ ]:
def xpath():
    start = time.time()
    OrderedDict({'Award/AwardID' : tree.xpath('Award/AwardID')[0].text,
                 'Award/AwardTitle' : tree.xpath('Award/AwardTitle')[0].text,
                 'Award/AwardEffectiveDate' : tree.xpath('Award/AwardEffectiveDate')[0].text,
                 'Award/AwardExpirationDate':tree.xpath('Award/AwardExpirationDate')[0].text,
                 'Award/AwardAmount':tree.xpath('Award/AwardAmount')[0].text,
                 'Award/Institution/Name' :tree.xpath('Award/Institution/Name')[0].text,
                 'Award/Organization/Division' : tree.xpath('Award/Organization/Division')[0].text[0],
                 'Award/Investigator/EmailAddress' : tree.xpath('Award/Investigator/EmailAddress')[0].text})
    return((time.time()-start)*1e6)

In [ ]:
f1 = 0
f2 = 0
x = 0
for i in range(10000):
    f1 += find1()
    f2 += find2()
    x += xpath()
print(f1/10000)
print(f2/10000)
print(x/10000)

In [ ]:
r = tree.xpath('Award/Institution/CityName')
print(len(r))
r[0].text

## Annual

In [13]:
# Initiate globals
def scrape(path):
    awards = []
    abstracts = []
    institutions = []
    PI = []
    
    # Builder functions
    def awards_builder(tree):
        return awards.append(
                OrderedDict([
                        ('AwardID',tree.find('Award/AwardID').text),
                        ('AwardTitle',tree.find('Award/AwardTitle').text),
                        ('AwardEffectiveDate',tree.find('Award/AwardEffectiveDate').text),
                        ('AwardExpirationDate',tree.find('Award/AwardExpirationDate').text),
                        ('AwardAmount',tree.find('Award/AwardAmount').text),
                        ('InstitutionName',tree.find('Award/Institution/Name').text),
                        ('Division',tree.find('Award/Organization/Division')[0].text),
                        ('EmailAddress',tree.find('Award/Investigator/EmailAddress').text)
                ]))
    def abstracts_builder(tree):
        return abstracts.append(
                OrderedDict([
                        ('AwardID',tree.find('Award/AwardID').text),
                        ('Abstract',tree.find('Award/AbstractNarration').text)
                 ]))
    def institutions_builder(tree):
        return institutions.append(
                OrderedDict([
                        ('InstitutionName',tree.find('Award/Institution/Name').text),
                        ('StreetAddress',tree.find('Award/Institution/StreetAddress').text),
                        ('CityName',tree.find('Award/Institution/CityName').text),
                        ('StateCode',tree.find('Award/Institution/StateCode').text),
                        ('ZipCode',tree.find('Award/Institution/ZipCode').text),
                        ('CountryName',tree.find('Award/Institution/CountryName').text)
                ]))
    def PI_builder(tree):
        return PI.append(
                OrderedDict([
                        ('EmailAddress',tree.find('Award/Investigator/EmailAddress').text),
                        ('FirstName',tree.find('Award/Investigator/FirstName').text),
                        ('LastName',tree.find('Award/Investigator/LastName').text)
                ]))

    def xml_parse(file):
        tree = ET.parse(file)
        # Builders
        try:
            awards_builder(tree)
        except:
            return(tree.find('Award/AwardID').text,"is missing a award key")
        abstracts_builder(tree)
        institutions_builder(tree)
        PI_builder(tree)
    
    def remove_duplicates(dict_list):
        return [dict(tupleized) for tupleized in set(tuple(item.items()) for item in dict_list)]
    
    def json_dump():
        dict_NSF = [awards,
                    abstracts,
                    institutions,
                    PI]
        
        outfiles = ["NSF_AWARDS_"+path+".json",
                    "NSF_ABSTRACTS_"+path+".json",
                    "NSF_INSTITUTIONS_"+path+".json",
                    "NSF_PI_"+path+".json"]
        
        for i in range(len(dict_NSF)):
            with open(outfiles[i], 'w') as outfile:
                json.dump(dict_NSF[i], outfile, indent=4)

    # main script
    print("***",path,"***")
    start = time.time()
    for file in os.listdir('data/'+path):
        xml_parse(os.path.join('data/'+path,file))
    print(len(awards),"records")
    print("Scraped in", round(time.time()-start, 2), "sec.")
    
    # remove doops
    PI = remove_duplicates(PI)
    institutions = remove_duplicates(institutions)
    
    # dump'em 
    start = time.time()
    json_dump()
    print("Dumped in",round(time.time()-start,2), "sec.") 
    #TODO: create a SQL or noSQL dump!

# iterate through each year
paths = [str(x) for x in range(2000,2017)]
for path in paths:
    scrape(path)

*** 2000 ***
10318 records
Scraped in 9.72 sec.
Dumped in 0.76 sec.
*** 2001 ***
9840 records
Scraped in 8.05 sec.
Dumped in 0.73 sec.
*** 2002 ***
10945 records
Scraped in 9.83 sec.
Dumped in 0.67 sec.
*** 2003 ***
11553 records
Scraped in 10.14 sec.
Dumped in 0.7 sec.
*** 2004 ***
10714 records
Scraped in 11.76 sec.
Dumped in 0.92 sec.
*** 2005 ***
10718 records
Scraped in 12.24 sec.
Dumped in 0.97 sec.
*** 2006 ***
11055 records
Scraped in 10.62 sec.
Dumped in 1.54 sec.
*** 2007 ***
12215 records
Scraped in 11.18 sec.
Dumped in 1.11 sec.
*** 2008 ***
12607 records
Scraped in 12.15 sec.
Dumped in 0.8 sec.
*** 2009 ***
15226 records
Scraped in 11.16 sec.
Dumped in 0.96 sec.
*** 2010 ***
13089 records
Scraped in 10.59 sec.
Dumped in 0.79 sec.
*** 2011 ***
11649 records
Scraped in 8.73 sec.
Dumped in 0.69 sec.
*** 2012 ***
12165 records
Scraped in 10.17 sec.
Dumped in 0.8 sec.
*** 2013 ***
11480 records
Scraped in 9.74 sec.
Dumped in 0.71 sec.
*** 2014 ***
12055 records
Scraped in 11.22

In [66]:
# todo fix ref b4 assign and where to init lists
# Initiate globals  
paths = [str(x) for x in range(2000,2017)]

def xml_scrape(file):
    tree = ET.parse(file)
    # Builders
    try:
        awards.append(
            OrderedDict([
                    ('AwardID',tree.find('Award/AwardID').text),
                    ('AwardTitle',tree.find('Award/AwardTitle').text),
                    ('AwardEffectiveDate',tree.find('Award/AwardEffectiveDate').text),
                    ('AwardExpirationDate',tree.find('Award/AwardExpirationDate').text),
                    ('AwardAmount',tree.find('Award/AwardAmount').text),
                    ('InstitutionName',tree.find('Award/Institution/Name').text),
                    ('Division',tree.find('Award/Organization/Division')[0].text),
                    ('EmailAddress',tree.find('Award/Investigator/EmailAddress').text)]))
    except:
        return(tree.find('Award/AwardID').text,"is missing a award key")

    abstracts.append(
            OrderedDict([
                    ('AwardID',tree.find('Award/AwardID').text),
                    ('Abstract',tree.find('Award/AbstractNarration').text)]))
    institutions.append(
            OrderedDict([
                    ('InstitutionName',tree.find('Award/Institution/Name').text),
                    ('StreetAddress',tree.find('Award/Institution/StreetAddress').text),
                    ('CityName',tree.find('Award/Institution/CityName').text),
                    ('StateCode',tree.find('Award/Institution/StateCode').text),
                    ('ZipCode',tree.find('Award/Institution/ZipCode').text),
                    ('CountryName',tree.find('Award/Institution/CountryName').text)]))
    PI.append(
            OrderedDict([
                    ('EmailAddress',tree.find('Award/Investigator/EmailAddress').text),
                    ('FirstName',tree.find('Award/Investigator/FirstName').text),
                    ('LastName',tree.find('Award/Investigator/LastName').text)]))

def remove_duplicates(dict_list):
    return [dict(tupleized) for tupleized in set(tuple(item.items()) for item in dict_list)]

def json_dump():
    dict_NSF = [awards,
                abstracts,
                institutions2,
                PI2]

    outfiles = ["NSF_AWARDS_"+path+".json",
                "NSF_ABSTRACTS_"+path+".json",
                "NSF_INSTITUTIONS_"+path+".json",
                "NSF_PI_"+path+".json"]

    for i in range(len(dict_NSF)):
        with open(outfiles[i], 'w') as outfile:
            json.dump(dict_NSF[i], outfile, indent=4)

def main():
    awards = []
    abstracts = []
    institutions = []
    PI = []
    # iterate through each year
    start = time.time()
    for path in paths:
        print("***",path,"***")
        for file in os.listdir('data/'+path):
            xml_scrape(os.path.join('data/'+path,file))
        print(len(awards),"records")
    print("Scraped in", round(time.time()-start, 2), "sec.")

    # remove doops
    PI2 = remove_duplicates(PI)
    institutions2 = remove_duplicates(institutions)

    # dump'em 
    start = time.time()
    json_dump()
    print("Dumped in",round(time.time()-start,2), "sec.") 
    #TODO: create a SQL or noSQL dump!
main()

*** 2000 ***
0 records
*** 2001 ***
0 records
*** 2002 ***
0 records
*** 2003 ***
0 records
*** 2004 ***


KeyboardInterrupt: 

## Interannual

In [61]:
# remove stuff I don't want...
rep = {"Division ": "", "Of ": "","of ": "", "Div ": "","Directorate ":"","Divn ":"","for ":""}
rep = dict((re.escape(k), v) for k, v in rep.items())
pattern = re.compile("|".join(rep.keys()))

def scrape(path):
    # Builder functions
    def awards_builder(tree):
        return awards.append(
                OrderedDict([
                        ('AwardID',tree.find('Award/AwardID').text),
                        ('AwardTitle',tree.find('Award/AwardTitle').text),
                        ('AwardEffectiveDate',tree.find('Award/AwardEffectiveDate').text),
                        ('AwardExpirationDate',tree.find('Award/AwardExpirationDate').text),
                        ('AwardAmount',tree.find('Award/AwardAmount').text),
                        ('InstitutionName',tree.find('Award/Institution/Name').text),
                        ('Division',pattern.sub(lambda m: rep[re.escape(m.group(0))],
                                                tree.find('Award/Organization/Division')[0].text)),
                        ('EmailAddress',tree.find('Award/Investigator/EmailAddress').text)
                ]))
    
    def abstracts_builder(tree):
        return abstracts.append(
                OrderedDict([
                        ('AwardID',tree.find('Award/AwardID').text),
                        ('Abstract',tree.find('Award/AbstractNarration').text)
                 ]))
   
    def institutions_builder(tree):
        return institutions.append(
                OrderedDict([
                        ('InstitutionName',tree.find('Award/Institution/Name').text),
                        ('StreetAddress',tree.find('Award/Institution/StreetAddress').text),
                        ('CityName',tree.find('Award/Institution/CityName').text),
                        ('StateCode',tree.find('Award/Institution/StateCode').text),
                        ('ZipCode',tree.find('Award/Institution/ZipCode').text),
                        ('CountryName',tree.find('Award/Institution/CountryName').text)
                ]))
    # what if two pis, what if pi moves universities?
    def PI_builder(tree):
        return PI.append(
                OrderedDict([
                        ('EmailAddress',tree.find('Award/Investigator/EmailAddress').text),
                        ('FirstName',tree.find('Award/Investigator/FirstName').text),
                        ('LastName',tree.find('Award/Investigator/LastName').text)
                ]))

    def xml_parse(file):
        tree = ET.parse(file)
        try:
            awards_builder(tree)
        except:
            return(tree.find('Award/AwardID').text,"is missing a award key")
        
        abstracts_builder(tree)
       
        institutions_builder(tree)

        PI_builder(tree)

    # main script
    print("***",path,"***")
    # iterate through the path directory
    for file in os.listdir('data/'+path):
        xml_parse(os.path.join('data/'+path,file))
    
def json_dump():
    dict_NSF = [awards,
                abstracts,
                institutions,
                PI]

    outfiles = ["NSF_AWARDS.json",
                "NSF_ABSTRACTS.json",
                "NSF_INSTITUTIONS.json",
                "NSF_PI.json"]

    for i in range(len(dict_NSF)):
        with open(outfiles[i], 'w') as outfile:
            json.dump(dict_NSF[i], outfile, indent=4)

def remove_duplicates(dict_list):
    return [dict(tupleized) for tupleized in set(tuple(item.items()) for item in dict_list)]
    
# iterate through each year
awards = []
abstracts = []
institutions = []
PI = []
# scrape the files
start = time.time()
for path in paths:
    scrape(path)
print(len(awards),"records")
print("Scraped in", round(time.time()-start, 2), "sec.")
print("****************")
# remove doops
start = time.time()
PI = remove_duplicates(PI)
institutions = remove_duplicates(institutions)
print("Duplicates removed in",round(time.time()-start,2), "sec.") 
# dump'em
start = time.time()
json_dump()
print("Dumped into JSON in",round(time.time()-start,2), "sec.") 
    #TODO: create a SQL or noSQL dump!

*** 2000 ***
*** 2001 ***
*** 2002 ***
*** 2003 ***
*** 2004 ***
*** 2005 ***
*** 2006 ***
*** 2007 ***
*** 2008 ***
*** 2009 ***
*** 2010 ***
*** 2011 ***
*** 2012 ***
*** 2013 ***
*** 2014 ***
*** 2015 ***
*** 2016 ***
185222 records
Scraped in 175.42 sec.
****************
Duplicates removed in 2.52 sec.
Dumped into JSON in 10.11 sec.


## Analysis of NSF 2015 
Cool things to look at
<br> Common words in titles
<br> email each PI with a status update.
<br> Chloropleth about instituional funding.

In [43]:
outfiles = ["NSF_AWARDS.json",
            "NSF_ABSTRACTS.json",
            "NSF_INSTITUTIONS.json",
            "NSF_PI.json"]
award_df = pd.read_json(outfiles[0])
abstract_df = pd.read_json(outfiles[1])
org_df = pd.read_json(outfiles[2])
pi_df = pd.read_json(outfiles[3])
pi_df['FullName']=pi_df.FirstName.str.lower()+" "+pi_df.LastName.str.lower()

Let's take a look at the columns...

In [44]:
len(award_df.index)

185222

For RDBA this works great... However, for publishing results nobody wants columns displayed_like_this! So here's a helper function to convert underscored columns into respectible, well-mannered headers. Appropriated from Stackoverflow user <a target="_blank" href='http://stackoverflow.com/a/6425628/5094480'>Siegfried Gevatter</a>

In [38]:
def beautify(underscored_word):
    # check for acronym
    if(len(underscored_word)>2):
        return ' '.join(x.capitalize() or '_' for x in underscored_word.split('_'))
    else:
        return underscored_word

In [18]:
def beautify1(camelCaseWord):
    # check for acronym
    return(''.join(map(lambda x: x if x.islower() else " "+x, camelCaseWord)))

A simple, flexible reporting function to find the top n funded column.

In [54]:
def top_funds(col,n):
    top_fund_col = [{beautify1(col):uni, "Total Award Money" : award_df.loc[award_df[col]==uni]['AwardAmount'].sum()} for uni in award_df[col].unique()]
    fund_df = pd.DataFrame(sorted(top_fund_col, key=lambda k: k['Total Award Money'], reverse=True))
    fund_df.index = fund_df.index+1
    return fund_df[:n]

In [56]:
def top_awarded(col,n):
    top_fund_col = [{beautify1(col):uni, "Total Awards" : len(award_df.loc[award_df[col]==uni])} for uni in award_df[col].unique()]
    fund_df = pd.DataFrame(sorted(top_fund_col, key=lambda k: k['Total Awards'], reverse=True))
    fund_df.index = fund_df.index+1
    return fund_df[:n]

In [47]:
top_funds('InstitutionName',5)

,Institution Name,Total Award Money
1,University Corporation For Atmospheric Res,1850184143
2,University of Illinois at Urbana-Champaign,1781954041
3,Cornell University,1607146837
4,University of Wisconsin-Madison,1521433540
5,University of California-Berkeley,1513344314


In [106]:
def pie():
    university_df = award_df[award_df.InstitutionName=="University of Washington"]
    d = pd.DataFrame(columns=("Division","Funding")
    for div in university_df.Division.unique():
        d[div]=university_df.AwardAmount[university_df.Division==div].sum()
    for key in d:
        print(key,d[key])

pie()

Empty DataFrame
Columns: [Division, Funding, Div Of Civil, Mechanical, & Manufact Inn, Div Of Electrical, Commun & Cyber Sys, Div Of Engineering Education and Centers, Division Of Behavioral and Cognitive Sci, Division Of Environmental Biology, Division Of Materials Research, Division Of Mathematical Sciences, Division Of Astronomical Sciences, Division Of Research On Learning, Division Of Integrative Organismal Sys, Division Of Earth Sciences, Division Of Chemistry, Divn Of Social and Economic Sciences, Div Of Chem, Bioeng, Env, & Transp Sys, Div Of Information & Intelligent Systems, Division Of Ocean Sciences, Division Of Computer and Network Systems, Division of Computing and Communication Foundations, Division Of Undergraduate Education, Division Of Polar Programs, Div Atmospheric & Geospace Sciences, Office of Integrative Activities, Div Of Biological Infrastructure, Emerging Frontiers & Multidisciplinary Activities, Emerging Frontiers, Div Of Molecular and Cellular Bioscience, Di

In [50]:
top_awarded('InstitutionName',5)

,Institution Name,Total Awards
1,University of Michigan Ann Arbor,2971
2,University of Washington,2703
3,University of Illinois at Urbana-Champaign,2680
4,University of California-Berkeley,2635
5,Pennsylvania State Univ University Park,2438


In [59]:
top_funds('Division',10)

,Division,Total Award Money
1,Ocean Sciences,5894117807
2,Astronomical Sciences,4419220783
3,Materials Research,4389509128
4,Research On Learning,4254154357
5,Physics,4101306480
6,Graduate Education,3772217183
7,Undergraduate Education,3761691226
8,Computer and Network Systems,3488000607
9,Chemistry,3338971470
10,Polar Programs,3329623418


In [58]:
top_awarded('Division',10)

,Division,Total Awards
1,Mathematical Sciences,15008
2,Earth Sciences,9429
3,Undergraduate Education,8562
4,Computer and Network Systems,8319
5,"Civil, Mechanical, & Manufact Inn",8194
6,Social and Economic Sciences,8167
7,"Chem, Bioeng, Env, & Transp Sys",7978
8,Chemistry,7660
9,Ocean Sciences,7491
10,Materials Research,7451


In [46]:
def report(InstitutionName,Division):
    return award_df[(award_df.InstitutionName==InstitutionName) & 
                    (award_df.Division==Division)].merge(abstract_df,on='AwardID').set_index('AwardID')[
                    ["AwardTitle","EmailAddress","Abstract"]]

report("University of Washington","Ocean Sciences")

,AwardTitle,EmailAddress,Abstract
AwardID,,,
717,Collaborative Research: A Sea Test of the PROD...,johnson@ocean.washington.edu,0000717<br/>Johnson<br/>This award to Oregon S...
735,NEP GLOBEC: Mesoscale Euphausiid and Hake Dist...,bhickey@u.washington.edu,This project addresses the Retrospective aspec...
990,Shipboard Scientific Support Equipment 2000,mcduff@ocean.washington.edu,"0000990 <br/>McDuff, Russell E.<br/><br/>This ..."
1034,GLOBEC: Long Term Monitoring and Analysis of C...,bhickey@u.washington.edu,GLOBEC Long-term Observation Program in the Ma...
2292,Fiber Optic Salinity/Density Probe,malford@ucsd.edu,This ocean science technology development proj...
2331,Measuring Vortex Force and Turbulence Fluxes i...,sanford@snark.apl.washington.edu,0002331 Lien/UW<br/><br/>The intent of this st...
2391,Mechanisms of Organic Matter Preservation in M...,jihedges@u.washington.edu,ABSTRACT<br/><br/>OCE-0002391<br/><br/><br/>In...
2628,Collaborative Research: Geodynamic Tomography:...,wilcock@u.washington.edu,This project will develop a suite of geophysic...
2684,"REVEL 2000 (Research and Education: Volcanoes,...",jdelaney@u.washington.edu,Delaney/00-02684: This award provides funding ...


In [27]:
def topics_finder(topic):
    return (award_df.loc[(award_df['AwardTitle'].notnull())&
                         (award_df['AwardTitle'].str.contains(topic,case=False))])
topics_finder("alzheimer")

,AwardAmount,AwardEffectiveDate,AwardExpirationDate,AwardID,AwardTitle,Division,EmailAddress,InstitutionName
26391,99513,09/01/2002,08/31/2003,225774,SGER: Assisted Cognition: First Steps Towards ...,Information & Intelligent Systems,etzioni@cs.washington.edu,University of Washington
137736,20000,08/15/2011,07/31/2013,1151033,EAGER: Evaluating small sugars as recognition ...,"Chemical, Bioengineering, Environmental, and T...",jehenry@lsu.edu,Louisiana State University & Agricultural and ...
166393,5070,06/01/2014,05/31/2015,1414629,EAPSI: Understanding the Role of the Protein C...,Office Internatl Science &Engineering,None,Perlenfein Tyler J
169696,224270,09/01/2014,11/30/2014,1435584,Collaborative Research: Data-Driven Smart Moni...,"Civil, Mechanical, & Manufact Inn",kewei.chen@bannerhealth.com,University of South Florida
169734,155729,09/01/2014,08/31/2017,1435809,Collaborative Research: Data-Driven Smart Moni...,"Civil, Mechanical, & Manufact Inn",kliu8@wisc.edu,University of Wisconsin-Madison
171889,100000,08/01/2014,07/31/2016,1447057,EAGER: Influence of the Alzheimer's Disease Mi...,"Chem, Bioeng, Env, & Transp Sys",jleach@umbc.edu,University of Maryland Baltimore County
176754,232270,09/01/2014,08/31/2017,1505260,Collaborative Research: Data-Driven Smart Moni...,"Civil, Mechanical, & Manufact Inn",shuai.huang.ie@gmail.com,University of Washington
178762,5070,06/01/2015,05/31/2016,1515384,EAPSI: Understanding Alzheimer's Disease throu...,Office Internatl Science &Engineering,None,Chen Kevin H


In [305]:
def PI_Report(PI):
    # search email of full name
    try:
        return award_df[award_df.EmailAddress.isin(pi_df.EmailAddress[pi_df.FullName==PI.lower()].values)]
        #return df.AwardTitle.values
    except:
        return(PI,"Not found in Our database")
    # search awards for email
    # display states
PI_Report('nicole lovenduski')

,AwardAmount,AwardEffectiveDate,AwardExpirationDate,AwardID,AwardTitle,Division,EmailAddress,InstitutionName
23323,453752,04/01/2012,03/31/2016,1155240,The Variable and Changing Carbonate Chemistry ...,Ocean Sciences,nicole.lovenduski@colorado.edu,University of Colorado at Boulder
35405,401386,06/01/2013,05/31/2016,1258995,Collaborative Research: Reacting Tracers in a ...,Ocean Sciences,nicole.lovenduski@colorado.edu,University of Colorado at Boulder


In [172]:
award_df[award_df.EmailAddress=='jig@ldeo.columbia.edu']

,AwardAmount,AwardEffectiveDate,AwardExpirationDate,AwardID,AwardTitle,Directorate,Division,EmailAddress,InstitutionName
17202,60124,10/04/2010,10/31/2012,1121022,Collaborative Research: Quantitative Importanc...,Directorate for Geosciences,Division of Ocean Sciences,jig@ldeo.columbia.edu,Columbia University
17209,107769,12/13/2010,12/31/2012,1121039,Collaborative Research: Grazing and Iron Contr...,Directorate for Geosciences,Division of Ocean Sciences,jig@ldeo.columbia.edu,Columbia University
19405,221690,02/01/2011,09/30/2014,1133277,Collaborative Research: ETBC: Amazon iNfluence...,Directorate For Geosciences,Division Of Ocean Sciences,jig@ldeo.columbia.edu,Columbia University


In [ ]:
# TODO fix PI shared names with email addresses!
def top_PI():
    award_df['PI'] = award_df.first_name+" "+award_df.last_name
    pi_df['PI'] = pi_df.first_name+" "+pi_df.last_name
    top_fund_col = []
    for PI in pi_df.PI:
        top_fund_col.append(
            {'PI':PI, 'funding_division': np.asarray(award_df[award_df.PI==PI].division)[0],"Total_Award_Money" : award_df[award_df['PI']==PI].award_amount.sum()})
    fund_df = pd.DataFrame(sorted(top_fund_col, key=lambda k: k['Total_Award_Money'], reverse=True))
    fund_df = pd.merge(left=fund_df,right=pi_df,on='PI',how='inner')
    fund_df.index = fund_df.index+1
    fund_df.rename(columns=lambda x: beautify(x), inplace=True)
    return fund_df[["Total Award Money","PI","Email Address","Funding Division"]][:25].drop_duplicates()
top_PI()